In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [54]:
data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
!wget $data -O scores.csv

--2025-10-15 23:56:51--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: 'scores.csv'

     0K .......... .......... .......... .......... .......... 63%  411K 0s
    50K .......... .......... ........                        100% 2.23M=0.1s

2025-10-15 23:56:52 (588 KB/s) - 'scores.csv' saved [80876/80876]



In [74]:
# importing and viewing dataset
df = pd.read_csv("scores.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [76]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [82]:
columns = df.columns
for col in columns:
    print(df[col].value_counts())
    print("Sum of null ",df[col].isnull().sum())
    print()

lead_source
organic_search    282
social_media      278
paid_ads          264
referral          260
events            250
Name: count, dtype: int64
Sum of null  128

industry
retail           203
finance          200
other            198
education        187
healthcare       187
technology       179
manufacturing    174
Name: count, dtype: int64
Sum of null  134

number_of_courses_viewed
1    417
2    388
3    269
0    181
4    109
5     67
6     22
7      6
8      2
9      1
Name: count, dtype: int64
Sum of null  0

annual_income
62478.0    2
62595.0    2
63133.0    2
74166.0    2
45569.0    2
          ..
69519.0    1
67832.0    1
63977.0    1
62536.0    1
47145.0    1
Name: count, Length: 1267, dtype: int64
Sum of null  181

employment_status
self_employed    352
student          348
unemployed       334
employed         328
Name: count, dtype: int64
Sum of null  100

location
north_america    225
europe           216
middle_east      198
asia             195
south_america    192
af

In [83]:
numerical = (['annual_income','lead_score', 'interaction_count', 'number_of_courses_viewed'])
categorical_columns = (['lead_source', 'industry','employment_status', 'location'])

In [84]:
df[numerical] = df[numerical].fillna(0.0)
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [85]:
df.converted.value_counts()

converted
1    905
0    557
Name: count, dtype: int64

In [86]:
df_full_train[categorical_columns].nunique()

lead_source          6
industry             8
employment_status    5
location             8
dtype: int64

### Setting up the validation framework

In [87]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [88]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [89]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [90]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

### EDA

In [91]:
df_full_train = df_full_train.reset_index(drop=True)

In [92]:
df_full_train.converted.value_counts(normalize=True)


converted
1    0.611634
0    0.388366
Name: proportion, dtype: float64

In [93]:
global_converted = df_full_train.converted.mean()

### Feature Importance

In [94]:
from IPython.display import display

In [95]:
for c in categorical_columns:
    print(c)
    df_group = df_full_train.groupby(c).converted.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_converted
    df_group['risk'] = df_group['mean'] / global_converted
    display(df_group)
    print()
    print()

lead_source


,mean,count,diff,risk
lead_source,,,,
NA,0.639175,97,0.027541,1.045029
events,0.604167,192,-0.007467,0.987791
organic_search,0.626087,230,0.014453,1.023630
paid_ads,0.439252,214,-0.172382,0.718162
referral,0.791262,206,0.179628,1.293686
social_media,0.591304,230,-0.020330,0.966762




industry


,mean,count,diff,risk
industry,,,,
NA,0.557522,113,-0.054112,0.911529
education,0.751592,157,0.139958,1.228827
finance,0.587097,155,-0.024537,0.959883
healthcare,0.597222,144,-0.014412,0.976437
manufacturing,0.642384,151,0.030750,1.050276
other,0.602564,156,-0.009070,0.985171
retail,0.581250,160,-0.030384,0.950323
technology,0.548872,133,-0.062762,0.897387




employment_status


,mean,count,diff,risk
employment_status,,,,
NA,0.534247,73,-0.077387,0.873474
employed,0.666667,273,0.055033,1.089977
self_employed,0.654545,275,0.042912,1.070159
student,0.659722,288,0.048088,1.078623
unemployed,0.476923,260,-0.134711,0.779753




location


,mean,count,diff,risk
location,,,,
NA,0.660000,50,0.048366,1.079077
africa,0.617450,149,0.005816,1.009509
asia,0.615894,151,0.004260,1.006965
australia,0.574324,148,-0.037310,0.939000
europe,0.636364,176,0.024730,1.040432
middle_east,0.630573,157,0.018939,1.030965
north_america,0.580645,186,-0.030989,0.949335
south_america,0.611842,152,0.000208,1.000340


### mutual information

In [96]:
from sklearn.metrics import mutual_info_score

In [97]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [98]:
mi = df_full_train[categorical_columns].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.024562
employment_status    0.012690
industry             0.008173
location             0.001212
dtype: float64

In [99]:
df_full_train[numerical].corrwith(df_full_train.converted).abs()

annual_income               0.068845
lead_score                  0.196394
interaction_count           0.387251
number_of_courses_viewed    0.434872
dtype: float64

### Correlation

In [100]:
corr_matrix = df[numerical].corr()
corr_matrix

,annual_income,lead_score,interaction_count,number_of_courses_viewed
annual_income,1.000000,0.015610,0.027036,0.009770
lead_score,0.015610,1.000000,0.009888,-0.004879
interaction_count,0.027036,0.009888,1.000000,-0.023565
number_of_courses_viewed,0.009770,-0.004879,-0.023565,1.000000


In [105]:
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [109]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [110]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)

C:\Users\RANDY\Documents\data_eng_project\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [111]:
model.intercept_[0]

np.float64(-2.335660627773248)

In [112]:
model.coef_[0].round(3)

array([-0.   , -0.437,  0.298, -0.358, -0.091, -1.738, -0.444,  0.319,
       -0.154, -0.447, -0.273, -0.617, -0.418, -0.293,  0.86 ,  0.131,
       -0.098, -0.285, -0.491, -1.64 ,  0.762, -0.574,  0.047, -0.304,
       -0.266, -0.449, -0.305, -0.42 , -0.53 , -0.099,  1.186])

In [113]:
y_pred = model.predict_proba(X_val)[:, 1]

In [117]:
converted_decision = (y_pred >= 0.5)

In [118]:
df_val[converted_decision].index

Index([  1,   2,   3,   4,   5,   6,   9,  10,  11,  12,
       ...
       281, 282, 283, 284, 285, 286, 288, 290, 291, 292],
      dtype='int64', length=191)

In [119]:
y_val

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1])

In [120]:
converted_decision.astype(int)

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1])

In [121]:
(y_val == converted_decision).mean()

np.float64(0.8020477815699659)

In [122]:
df_pred = pd.DataFrame()
df_pred['probablilty'] = y_pred
df_pred['prediction'] = converted_decision.astype(int)
df_pred['actual'] = y_val

In [124]:
df_pred

,probablilty,prediction,actual
0,0.074400,0,1
1,0.532989,1,1
2,0.690881,1,1
3,0.744288,1,0
4,0.784425,1,1
...,...,...,...
288,0.717735,1,1
289,0.393091,0,1
290,0.997183,1,1
291,0.687472,1,0


In [135]:
pd.Series(model.coef_[0], index=dv.columns).sort_values(ascending=False)


AttributeError: 'DictVectorizer' object has no attribute 'columns'

In [127]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_val) 
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8020477815699659


In [133]:
# First, verify the shapes
print(f"Coefficients shape: {model.coef_[0].shape}")
print(f"df_train columns: {len(df_train.columns)}")
print(f"df_train shape: {df_train.shape}")

# If you used get_dummies or OneHotEncoder
print(f"Columns after encoding: {list(df_train.columns)}")

Coefficients shape: (31,)
df_train columns: 8
df_train shape: (876, 8)
Columns after encoding: ['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income', 'employment_status', 'location', 'interaction_count', 'lead_score']
